In [2]:
from IPython.core.display import *
from StringIO import StringIO
import csv
import operator, random, math

In [3]:
data = []
classes = ['e', 'p']
attributes = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 
              'odor', 'gill-attachment', 'gill-spacing', 'gill-size',
              'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
              'stalk-surface-below-ring', 'stalk-color-above-ring', 
              'stalk-color-below-ring', 'veil-type', 'veil-color',
              'ring-number', 'ring-type', 'spore-print-color',
              'population', 'habitat']
attribute_values = {'cap-shape':['b','c','x','f','k','s'], 
                  'cap-surface':['f','g','y','s'], 
                  'cap-color':['n','b','c','g','r','p','u','e','w','y'], 
                  'bruises':['t','f'], 
                  'odor':['a','l','c','y','f','m','n','p','s'],
                  'gill-attachment':['a','d','f','n'],
                  'gill-spacing':['c','w','d'],
                  'gill-size':['b','n'],
                  'gill-color':['k','n','b','h','g','r','o','p','u','e','w','y'],
                  'stalk-shape':['e','t'],
                  'stalk-root':['b','c','u','e','z','r','?'],
                  'stalk-surface-above-ring':['f','y','k','s'],
                  'stalk-surface-below-ring':['f','y','k','s'],
                  'stalk-color-above-ring':['n','b','c','g','o','p','e','w','y'],
                  'stalk-color-below-ring':['n','b','c','g','o','p','e','w','y'],
                  'veil-type':['p','u'],
                  'veil-color':['n','o','w','y'],
                  'ring-number':['n','o','t'],
                  'ring-type':['c','e','f','l','n','p','s','z'],
                  'spore-print-color':['k','n','b','h','r','o','u','w','y'],
                  'population':['a','c','n','s','v','y'],
                  'habitat':['g','l','m','p','u','w','d']
                 }
with open('agaricus-lepiota.data', 'r') as csvfile:
    datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in datareader:
        inputs = [v for v in row]
        d = {}
        d['class'] = inputs[0]
        index = 1
        for a in attributes:
            d[a] = inputs[index]
            index += 1
        data.append(d)
        
random.shuffle(data)

In [4]:
n = len(data)/2
data1 = data[0:n]
data2 = data[n:]

In [5]:
def product(probabilities):
    return reduce(operator.mul, probabilities)

In [6]:
def train(data, classes, attrs, attrvals):
    trained_data = {}
    for c in classes:
        trained_data[c] = {}
        for a in attrs:
            trained_data[c][a] = {}
            for v in attrvals[a]:
                trained_data[c][a][v] = 0
                trained_data[c][a]['all'] = 0
    for d in data:
        for a in attrs:
            trained_data[d['class']][a][d[a]] += 1
            trained_data[d['class']][a]['all'] += 1
    return trained_data

In [7]:
def probability(data, label, attr, attrval):
    return ((data[label][attr][attrval] + 1.0) / (data[label][attr]['all'] + 1.0))

In [8]:
def class_probability(data, classes):
    count = {}
    for c in classes:
        count[c] = 0
        count['all'] = 0
    for d in data:
        count[d['class']] += 1
        count['all'] += 1
    p = {}
    for c in classes:
        p[c] = float(count[c])/count['all']
    return p

In [9]:
def estimate(data, pc, data_sample):
    possible_classes = [c for c in data]
    best_class = possible_classes[0]
    possible_attrs = [a for a in trained_data1[best_class]]
    best_probability = pc[best_class]*product([probability(data, best_class, attr, data_sample[attr])
                                               for attr in possible_attrs])
    for c in possible_classes:
        possible_attrs = [a for a in trained_data1[c]]
        p = pc[c]*product([probability(data, c, attr, data_sample[attr])
                               for attr in possible_attrs])
        if p > best_probability:
            best_class = c
            best_probability = p
    return best_class

In [10]:
def classify(data, pc, test_data):
    result = []
    for d in test_data:
        result.append(estimate(data, pc, d))
    return result

In [11]:
def evaluate(test_data, classified_data):
    errors = 0
    n = len(test_data)
    for i in range(n):
        if test_data[i]['class'] != classified_data[i]:
            errors += 1
    error_rate = float(errors)/n
    return error_rate

In [12]:
trained_data1 = train(data1, classes, attributes, attribute_values)
pc1 = class_probability(data1, classes)

In [13]:
#if odor is creosote, the probability of being poisonous is much higher than edible
print probability(trained_data1, 'e', 'odor', 'c')
print probability(trained_data1, 'p', 'odor', 'c')

0.000466417910448
0.053125


In [14]:
classified_data1 = classify(trained_data1, pc1, data2)
for i in range(len(data2)):
    if classified_data1[i] == data2[i]['class']:
        print classified_data1[i], data2[i]['class']
    else:
        print classified_data1[i], data2[i]['class'], '<- error'
evaluate(data2, classified_data1)

p p
p p
e e
e e
e e
e e
e p <- error
p p
p p
p p
p p
e e
e e
p p
p p
e e
p p
p p
e e
e e
e e
e e
e e
p p
e e
e e
e e
p e <- error
p p
e e
p p
e e
e e
p p
e e
p p
p p
e e
p p
e e
e e
p p
p p
e p <- error
p p
p p
p p
p p
e e
e p <- error
p p
e e
e e
p p
e e
e e
e e
p p
p p
p p
p p
p p
e e
p p
p p
e e
p p
e e
e e
e e
e e
e e
p p
p p
p p
p p
e e
p p
p p
p p
e e
e e
e e
e e
p p
e e
p p
e e
e e
e e
p p
e e
e e
e e
p p
p p
e e
e e
p p
e e
e e
e e
p p
p p
e e
p p
p e <- error
e e
e e
p p
e e
e e
e e
e p <- error
e e
e e
e e
e p <- error
e e
e e
e e
p p
e e
e e
e e
p p
p p
e e
p p
p p
p p
p p
e e
e e
p p
e e
p p
e e
e e
e e
p e <- error
e e
e e
p p
e e
e e
e e
p p
e e
e p <- error
p p
p p
e e
e e
p p
p p
p p
p p
p p
p p
p p
e e
e e
e e
e e
e p <- error
p p
e e
e e
p p
p p
p p
e e
p p
p p
e e
p p
e e
p p
e p <- error
e e
p p
e e
p p
p p
p p
e e
e e
e e
p p
e e
e e
p p
p p
e p <- error
p p
e e
p p
p p
e e
e e
p p
p p
e e
e e
p p
e e
p p
e e
p p
p p
e e
e e
p p
e e
p p
p p
e e
e e
e e
p p
e e
p p


0.051698670605613

In [15]:
trained_data2 = train(data2, classes, attributes, attribute_values)
pc2 = class_probability(data2, classes)

In [16]:
classified_data2 = classify(trained_data2, pc2, data1)
for i in range(len(data1)):
    print classified_data2[i], data1[i]['class']
evaluate(data1, classified_data2)

e e
e e
p p
p p
e e
p p
p p
p p
p p
e e
e e
p p
e e
p p
p p
p p
e e
e e
e e
p p
p p
e e
p p
e p
e e
e e
e e
e e
e e
e p
p p
p p
e e
e e
e e
p p
e e
e e
p p
e e
e e
e e
p p
p p
e e
p p
e e
p p
e e
p p
p p
e e
e e
e e
e e
e e
e e
p p
e e
e e
p p
e e
e e
p p
p p
p p
e e
e e
e e
e e
e e
e e
e e
p p
e e
p p
e e
p p
e e
p p
e e
e e
e e
p p
p p
e e
e e
p p
e e
p p
e e
p p
e e
e e
p p
e e
e e
e e
e e
e e
e p
e e
p p
p p
p p
e e
p p
p p
e e
e e
e e
p p
p p
p p
e p
p p
e e
e p
e e
e e
e p
p p
p p
e e
p p
p p
e e
e e
e e
e e
p p
e e
p p
p p
p p
p p
e e
p p
p p
p p
e e
e e
e e
p p
e p
e e
p p
e e
e e
e e
e e
e e
e e
e e
p p
e e
p p
e e
e e
e e
p p
e e
p p
p p
p p
e e
p p
e e
p p
e e
e p
p p
e e
e e
e e
e e
p p
p p
e e
p p
e e
p p
p p
e p
e e
e e
e e
p p
e e
e e
p p
e e
p p
p p
e e
p p
p p
e p
e e
e e
e e
e p
p p
e e
e e
e e
p p
p p
e e
p p
e e
e e
e e
p p
e e
e e
e e
p p
p p
e e
e e
e e
e e
e e
e e
p p
p p
p p
e e
e e
p p
p p
e e
p p
e e
p p
p p
p p
e e
e e
e e
p p
p p
p p
p p
p p
e e
p p
p p
e e


0.05243722304283604